In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import json
import os
import cv2
import matplotlib.pyplot as plt
import shutil

##### import coco anotations

In [3]:
path = ''
!unzip "{path}"coco_data.zip -d "/content/coco-data"

Archive:  /content/drive/Shareddrives/P&I/Data & ML/PV&rooftop_project/coco_data.zip
  inflating: /content/coco-data/coco_data.json  
  inflating: /content/coco-data/__MACOSX/._coco_data.json  


In [4]:
f = open('/content/coco-data/coco_data.json')
coco_data = json.load(f)
f.close()
print(type(coco_data), len(coco_data))
coco_data.keys()

<class 'dict'> 6


dict_keys(['info', 'licenses', 'images', 'annotations', 'categories', 'tag_categories'])

In [ ]:
coco_data['categories']

[{'id': 0, 'name': 'Wintergarten ceiling', 'supercategory': 'root'},
 {'id': 1, 'name': 'PV', 'supercategory': 'root'},
 {'id': 2, 'name': 'Roof type', 'supercategory': 'root'},
 {'id': 3, 'name': 'Roof windows', 'supercategory': 'root'},
 {'id': 4, 'name': 'Dormers', 'supercategory': 'root'},
 {'id': 5, 'name': 'Thermal panels', 'supercategory': 'root'},
 {'id': 6, 'name': 'PV_Negative', 'supercategory': 'root'}]

##### convert coco to yolo: https://medium.com/red-buffer/converting-a-custom-dataset-from-coco-format-to-yolo-format-6d98a4fd43fc

In [ ]:
input_path = '/content/images/'
output_path = '/content/yolo_data/'

!mkdir yolo_data
!mkdir yolo_data/images
!mkdir yolo_data/labels

In [ ]:
len(os.listdir('/content/images'))

1000

In [ ]:
file_names = []

def load_images_from_folder(input_path, output_path):
  count = 0
  for filename in os.listdir(input_path):
        source = os.path.join(input_path,filename)
        destination = f"{output_path}images/{filename}"

        try:
            shutil.copy(source, destination)
            print("File copied successfully.")
        # If source and destination are same
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        file_names.append(filename)
        count += 1

load_images_from_folder(input_path, output_path)

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied 

In [ ]:
def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in coco_data['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None

def get_img(filename):
  for img in coco_data['images']:
    if img['file_name'] == filename:
      return img

In [ ]:
count = 0

for filename in file_names:
  # extract image
  img = get_img(filename)
  img_id = img['id']
  img_w = img['width']
  img_h = img['height']

  # get annotations for this image
  img_ann = get_img_ann(img_id)

  if img_ann:
    # open current image file
    imgname = filename.split(".")[0]
    file_object = open(f"{output_path}labels/{imgname}.txt", "a")

    for ann in img_ann:
      current_category = ann['category_id'] # - 1 # As yolo format labels start from 0


      # 'segmentation' is xy pairs of each point of the polygon...
      current_segm = ann['segmentation'][0]
      xs = current_segm[0::2]
      ys = current_segm[1::2]

      assert(len(xs)==len(ys))

      # normalization
      xs = [i / img_w for i in xs]
      ys = [i / img_h for i in ys]

      normed_segm = []
      for i in range(len(xs)):
        normed_segm.append(xs[i])
        normed_segm.append(ys[i])

      normed_segm = [format(i, '.3f') for i in normed_segm]

      # write current object
      output = f"{current_category} "
      for i in normed_segm:
        output += i
        output += " "
      output = output[:-1]

      file_object.write(f"{output}\n")
    file_object.close()
    count += 1

In [ ]:
len(os.listdir("/content/yolo_data/labels")), len(os.listdir("/content/yolo_data/images"))

(1000, 1000)

In [ ]:
for im in coco_data['images']:
  if im['file_name'] == '710.png':
    print(im['id'])

448


In [ ]:
for n in coco_data['annotations']:
  if n['image_id']==807:
    print(n)

{'id': 11682, 'image_id': 807, 'category_id': 2, 'segmentation': [[63.63, 173.83, 254.67, 74.64, 321.01, 206.48, 137.58, 297.24]], 'area': 30557.88824999999, 'bbox': [63.63, 74.64, 257.38, 222.60000000000002], 'iscrowd': 0, 'extra': {'attributes': ['pointed']}}
{'id': 11683, 'image_id': 807, 'category_id': 2, 'segmentation': [[3.74, 229.67, 50.39, 315.76, 0.03, 341.77, 0, 230.74]], 'area': 2982.071600000001, 'bbox': [0.0, 229.67, 50.39, 112.1], 'iscrowd': 0, 'extra': {'attributes': ['flat']}}


In [ ]:
with open('yolo_data/labels/710.txt', 'r') as f:
  text = f.read()
print(text)

1 0.385 0.374 0.369 0.514 0.499 0.528 0.513 0.386
1 0.551 0.390 0.538 0.496 0.591 0.502 0.597 0.431 0.622 0.432 0.614 0.503 0.664 0.512 0.677 0.406
3 0.638 0.957 0.634 0.983 0.656 0.985 0.658 0.958
3 0.703 0.966 0.697 0.990 0.719 0.994 0.723 0.968
3 0.743 0.974 0.740 0.996 0.753 0.996 0.755 0.973
3 0.601 0.452 0.599 0.478 0.615 0.479 0.617 0.453
3 0.512 0.444 0.509 0.471 0.539 0.474 0.540 0.446
3 0.460 0.295 0.454 0.318 0.469 0.321 0.474 0.296
3 0.513 0.291 0.508 0.315 0.525 0.317 0.527 0.292
3 0.620 0.305 0.620 0.327 0.632 0.326 0.632 0.305
5 0.907 0.415 0.903 0.472 1.000 0.481 1.000 0.427
3 0.092 0.921 0.091 0.947 0.110 0.947 0.110 0.920
2 0.442 0.132 0.431 0.226 0.415 0.223 0.406 0.313 0.388 0.312 0.381 0.343 0.327 0.318 0.350 0.120
2 0.680 0.530 0.574 0.520 0.534 0.546 0.488 0.546 0.484 0.539 0.363 0.529 0.377 0.356 0.380 0.343 0.381 0.343 0.388 0.312 0.406 0.313 0.415 0.224 0.422 0.225 0.431 0.226 0.431 0.225 0.440 0.226 0.464 0.214 0.563 0.225 0.589 0.244 0.710 0.259
2 0.760 0.37

In [ ]:
!zip -r yolo_segmentation_data.zip yolo_data/

  adding: yolo_data/ (stored 0%)
  adding: yolo_data/images/ (stored 0%)
  adding: yolo_data/images/972.png (deflated 0%)
  adding: yolo_data/images/576.png (deflated 0%)
  adding: yolo_data/images/85.png (deflated 0%)
  adding: yolo_data/images/934.png (deflated 0%)
  adding: yolo_data/images/599.png (deflated 0%)
  adding: yolo_data/images/137.png (deflated 0%)
  adding: yolo_data/images/271.png (deflated 0%)
  adding: yolo_data/images/31.png (deflated 0%)
  adding: yolo_data/images/323.png (deflated 0%)
  adding: yolo_data/images/989.png (deflated 0%)
  adding: yolo_data/images/996.png (deflated 0%)
  adding: yolo_data/images/8.png (deflated 0%)
  adding: yolo_data/images/283.png (deflated 0%)
  adding: yolo_data/images/647.png (deflated 0%)
  adding: yolo_data/images/747.png (deflated 0%)
  adding: yolo_data/images/116.png (deflated 0%)
  adding: yolo_data/images/798.png (deflated 0%)
  adding: yolo_data/images/348.png (deflated 0%)
  adding: yolo_data/images/225.png (deflated 0%)
